In [1]:
import xml.etree.ElementTree as ET
import requests

In [2]:
# nome dos ficheiros a criar

f = open('flickr.csv', 'w', encoding="utf-8")
g = open('smallboxes.csv', 'w', encoding="utf-8")

In [3]:
#create the cursor
#cur = conn.cursor
url = 'https://api.flickr.com/services/rest/?method=flickr.photos.search&api_key=d827ec49d172cace3be06ec605a30cb0&per_page=250'

In [4]:
##Coordenadas do rectangulo que abrange a area de estudo, graus decimais WGS84 
bboxlist = [[-84.689204,42.640682,-84.411112,42.800595]]
#initialize the smallbox list
smallboxes = []

In [5]:
#current box check for the total number of photos
def check_current_box(curbox):
    stringBox = str(curbox).strip("[]")
    args = { 'min_taken_date' : '2016-01-01', # data de inicio de recolha dos dados
             'max_taken_date' : '2021-09-29', # data de fim de recolha de dados
             'bbox' : stringBox,
             'extras' : 'geo, url_m, tags, views, date_taken, date_upload, media, owner_name'}
    box = requests.get(url, params = args)
    #need to parse element 'photos' in order to get
    #this will need to be debugged
    checkBox = ET.fromstring(box.content)
    total = int(checkBox[0].attrib['total'])
    nopages = int(checkBox[0].attrib['pages'])
    return {"total": total, "pages": nopages}

In [6]:
#function to get small enough boxes
def get_small_boxes(boundingbox):
    while len(boundingbox) > 0:
        for e in boundingbox:
            #get the current box
            currentbox = e
            #insert check_current_box here
            check = check_current_box(currentbox)
            nophotos, nopages = check['total'], check['pages']
            if nophotos >= 501:
                print ("[*] Too many photos in the current box %s, %s, %s" % (currentbox, nophotos, nopages) )
                #find the height of the current box
                h = (abs(e[3] - e[1]) * .5)
                #find the width of the current box
                w = (abs(e[0] - e[2]) * .5)
                new_lat = e[1] + h
                new_lon = e[0] + w
                nb1 = [e[0], new_lat, new_lon, e[3]]
                bboxlist.append(nb1)
                nb2 = [new_lon, new_lat, e[2], e[3]]
                bboxlist.append(nb2)
                nb3 = [e[0], e[1], new_lon, new_lat]
                bboxlist.append(nb3)
                nb4 = [new_lon, e[1], e[2], new_lat]
                bboxlist.append(nb4)
                index = boundingbox.index(e)
                bboxlist.pop(index)
            else:
                print ("[*] This box is small enough %s, %s, %s" % (currentbox, nophotos, nopages) )
                minilist = [e, nopages]
                smallboxes.append(minilist)
                get_data(e, nopages)
                index = boundingbox.index(e)
                bboxlist.pop(index)

def get_data(box, pages):
    curpage = 1
    while curpage <= pages:
        stringBox = str(box).strip("[]")
        args = {'bbox' : stringBox,
                    'min_taken_date' : '2016-01-01', # data de inicio de recolha dos dados
                    'max_taken_date' : '2021-09-29', # data de fim de recolha de dados
                    'extras' : 'geo, url_m, tags, views, date_taken, date_upload, media'}
        result = requests.get(url, params = args)
        data = ET.fromstring(result.content)
        curpage += 1
        for photo in data[0]:
            try:
                pid = str(photo.attrib['id'])
            except:
                pid = "PID problem"
            try:
                owner = str(photo.attrib['owner'])
            except:
                owner  = "owner problem"
            try:
                secret = str(photo.attrib['secret'])
            except:
                secret = "secret problem"
            try:
                server = str(photo.attrib['server'])
            except:
                server = "sever problem"
            try:
                farm = str(photo.attrib['farm'])
            except:
                farm = "farm problem"
            try:
                latitude = str(photo.attrib['latitude'])
            except:
                latitude = "latitude problem"
            try:    
                longitude = str(photo.attrib['longitude'])
            except:
                longitude = "longitude problem"
            try:
                woeid = str(photo.attrib['woeid'])
            except:
                woeid = "woied problem"
            try:
                place_id = str(photo.attrib['place_id'])
            except:
                place_id = "place_id problem"
            try:
                url_m =  str(photo.attrib['url_m'])
            except:
                url_m = "url_m problem"
            try:
                tags = str(photo.attrib['tags'])
            except:
                tags = "tags problem"
            try:
                views = str(photo.attrib['views'])
            except:
                views = "views problem"
            try:
                date_t = str(photo.attrib['datetaken'])
            except:
                date_t = "date_t problem"
            try:
                date_g = str(photo.attrib['datetakengranularity'])
            except:
                date_g = "date_g problem"        
            try:
                date_u = str(photo.attrib['dateupload'])
            except:
                date_u = "date_u problem"
            try:
                media = str(photo.attrib['media'])
            except:
                media = "media problem"
            try:
                title =str(photo.attrib['title'])
            except:
                title = "title problem"                
    
            line = "%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s\n" %(   pid,            
                                                                               owner,
                                                                               secret,
                                                                               server,
                                                                               farm,
                                                                               latitude, 
                                                                               longitude,
                                                                               woeid,
                                                                               place_id,
                                                                               url_m,
                                                                               tags,
                                                                               views,
                                                                               date_t,
                                                                               date_g,
                                                                               date_u,
                                                                               media,
                                                                               title)
            
            f.write(line)
            print ("writing line")

get_small_boxes(bboxlist)


#this writes the small boxes list to a file.
for box in smallboxes:
    sbox = str(box)
    g.write(sbox + '"\n')

[*] Too many photos in the current box [-84.689204, 42.640682, -84.411112, 42.800595], 25074, 101
[*] Too many photos in the current box [-84.55015800000001, 42.7206385, -84.411112, 42.800595], 23458, 94
[*] Too many photos in the current box [-84.55015800000001, 42.640682, -84.411112, 42.7206385], 1325, 6
[*] This box is small enough [-84.480635, 42.76061675, -84.411112, 42.800595], 4, 1
writing line
writing line
writing line
[*] Too many photos in the current box [-84.480635, 42.7206385, -84.411112, 42.76061675], 7363, 30
[*] This box is small enough [-84.480635, 42.68066025, -84.411112, 42.7206385], 341, 2
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writin

[*] Too many photos in the current box [-84.55015800000001, 42.68066025, -84.480635, 42.7206385], 791, 4
[*] Too many photos in the current box [-84.480635, 42.740627625, -84.4458735, 42.76061675], 699, 3
[*] This box is small enough [-84.689204, 42.76061675, -84.61968100000001, 42.800595], 11, 1
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
[*] This box is small enough [-84.689204, 42.7206385, -84.61968100000001, 42.76061675], 111, 1
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing l

[*] This box is small enough [-84.55015800000001, 42.68066025, -84.51539650000001, 42.700649375], 390, 2
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing lin

writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line

[*] This box is small enough [-84.46325425, 42.7506221875, -84.4458735, 42.76061675], 29, 1
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
[*] Too many photos in the current box [-84.55015800000001, 42.7206385, -84.480635, 42.76061675], 15652, 63
[*] Too many photos in the current box [-84.61968100000001, 42.7206385, -84.55015800000001, 42.76061675], 1483, 6
[*] Too many photos in the current box [-84.46325425, 42.740627625, -84.4458735, 42.7506221875], 575, 3
[*] This box is small enough [-84.51539650000001, 42.740627625, -84.480635, 42.76061675], 78, 1
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writing line
writi

...

[*] Too many photos in the current box [-84.54612046345746, 42.734740911222076, -84.54612045516967, 42.73474091598785], 715, 3
[*] Too many photos in the current box [-84.54612046345746, 42.734740906456295, -84.54612045516967, 42.734740911222076], 715, 3
[*] Too many photos in the current box [-84.54612046345746, 42.734740901690515, -84.54612045516967, 42.734740906456295], 712, 3
[*] Too many photos in the current box [-84.54612046345746, 42.734740896924734, -84.54612045516967, 42.734740901690515], 688, 3
[*] Too many photos in the current box [-84.54612046345746, 42.73474089215895, -84.54612045516967, 42.734740896924734], 702, 3
[*] Too many photos in the current box [-84.54612046345746, 42.73474088739317, -84.54612045516967, 42.73474089215895], 696, 3
[*] Too many photos in the current box [-84.54612046345746, 42.73474088262739, -84.54612045516967, 42.73474088739317], 705, 3
[*] Too many photos in the current box [-84.54612046345746, 42.73474087786162, -84.54612045516967, 42.73474088

[*] Too many photos in the current box [-84.54612046345746, 42.734740601446454, -84.54612045516967, 42.73474060621223], 697, 3
[*] Too many photos in the current box [-84.54612046345746, 42.73474059668068, -84.54612045516967, 42.734740601446454], 705, 3
[*] Too many photos in the current box [-84.54612046345746, 42.7347405919149, -84.54612045516967, 42.73474059668068], 715, 3
[*] Too many photos in the current box [-84.54612046345746, 42.73474058714912, -84.54612045516967, 42.7347405919149], 676, 3
[*] Too many photos in the current box [-84.54612046345746, 42.73474058238334, -84.54612045516967, 42.73474058714912], 715, 3
[*] Too many photos in the current box [-84.54612046345746, 42.73474057761756, -84.54612045516967, 42.73474058238334], 658, 3
[*] Too many photos in the current box [-84.54612046345746, 42.734740572851784, -84.54612045516967, 42.73474057761756], 697, 3
[*] Too many photos in the current box [-84.54612046345746, 42.73474056808601, -84.54612045516967, 42.734740572851784

[*] Too many photos in the current box [-84.54612045931356, 42.73474083258672, -84.54612045516967, 42.734740834969614], 725, 3
[*] Too many photos in the current box [-84.54612045931356, 42.73474083020383, -84.54612045516967, 42.73474083258672], 709, 3
[*] Too many photos in the current box [-84.54612045931356, 42.73474082782094, -84.54612045516967, 42.73474083020383], 698, 3
[*] Too many photos in the current box [-84.54612045931356, 42.73474082543805, -84.54612045516967, 42.73474082782094], 699, 3
[*] Too many photos in the current box [-84.54612045931356, 42.73474082305516, -84.54612045516967, 42.73474082543805], 699, 3
[*] Too many photos in the current box [-84.54612045931356, 42.73474082067227, -84.54612045516967, 42.73474082305516], 711, 3
[*] Too many photos in the current box [-84.54612045931356, 42.73474081828938, -84.54612045516967, 42.73474082067227], 716, 3
[*] Too many photos in the current box [-84.54612045931356, 42.73474081590649, -84.54612045516967, 42.73474081828938]

[*] Too many photos in the current box [-84.54612045931356, 42.73474067531602, -84.54612045516967, 42.73474067769891], 707, 3
[*] Too many photos in the current box [-84.54612045931356, 42.73474067293313, -84.54612045516967, 42.73474067531602], 671, 3
[*] Too many photos in the current box [-84.54612045931356, 42.73474067055024, -84.54612045516967, 42.73474067293313], 689, 3
[*] Too many photos in the current box [-84.54612045931356, 42.73474066816735, -84.54612045516967, 42.73474067055024], 702, 3
[*] Too many photos in the current box [-84.54612045931356, 42.73474066578446, -84.54612045516967, 42.73474066816735], 714, 3
[*] Too many photos in the current box [-84.54612045931356, 42.73474066340157, -84.54612045516967, 42.73474066578446], 698, 3
[*] Too many photos in the current box [-84.54612045931356, 42.73474066101868, -84.54612045516967, 42.73474066340157], 707, 3
[*] Too many photos in the current box [-84.54612045931356, 42.73474065863579, -84.54612045516967, 42.73474066101868],

ParseError: not well-formed (invalid token): line 17, column 81 (<string>)